# Neural Networks Implementation

### PreProcess Data

In [1]:
import PreProcessData as pre
import imp
imp.reload(pre)
X_train, X_valid, y_train, y_valid = pre.PreProcessData()

/tmp/ipykernel_1911389/4036690796.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


### Class Construction

In [56]:
import numpy as np
import pandas as pd

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.biases_input_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.biases_hidden_output = np.zeros((1, output_size))

    # TODO NEED Softmax function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def feedforward(self, X):
        # Input to hidden layer
        self.hidden_sum = np.dot(X, self.weights_input_hidden) + self.biases_input_hidden
        self.hidden_activation = self.sigmoid(self.hidden_sum)

        # Hidden to output layer
        self.output_sum = np.dot(self.hidden_activation, self.weights_hidden_output) + self.biases_hidden_output
        self.output_activation = self.sigmoid(self.output_sum)

        return self.output_activation

    def backward(self, X, y, output):
        # Output layer
        error_output = y - output
        delta_output = error_output * self.sigmoid_derivative(output)

        # Hidden layer
        # error_hidden = delta_output.dot(self.weights_hidden_output.T)
        error_hidden = error_output.dot(self.weights_hidden_output.T)
        # error_hidden = error_output.dot(self.hidden_activation.T)
        # TODO Make the matricies match each other
        # delta_hidden = error_hidden * self.sigmoid_derivative(self.hidden_activation.dot(self.weights_hidden_output))
        delta_hidden = error_hidden * self.sigmoid_derivative(self.hidden_activation)
        # print(self.hidden_activation.T.shape)
        # print(delta_output.shape)
        # print(delta_hidden.shape)
        # print(error_hidden.shape)

        # Update weights and biases
        # self.weights_hidden_output += self.hidden_activation.T.dot(delta_output) * self.learning_rate
        self.weights_hidden_output += self.hidden_activation.T.dot(error_output) * self.learning_rate
        self.biases_hidden_output += np.sum(error_output, axis=0, keepdims=True) * self.learning_rate
        self.weights_input_hidden += X.T.dot(delta_hidden) * self.learning_rate
        self.biases_input_hidden += np.sum(delta_hidden, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs):
        y = y.reshape(-1,1)
        for epoch in range(epochs):
            # Forward pass
            output = self.feedforward(X)

            # Backpropagation
            self.backward(X, y, output)

            # Print loss every 100 epochs
            if (epoch + 1) % 100 == 0:
                loss = np.mean(np.square(y - output))
                print(f'Epoch {epoch + 1}, Loss: {loss:.4f}')

    def predict(self, X):
        return self.feedforward(X)

    def validate(self, y_pred, y_valid):
        num_of_correct_predictions = np.sum((y_pred >= 0.5) == y_valid)
        accuracy = num_of_correct_predictions / len(y_valid)
        return accuracy

### Model Construction

In [3]:
# Create and train the neural network
input_size = X_train.shape[1]
hidden_size = 4 # Number of neurons in the hidden layer
# output_size = X_train.shape[0]
output_size = 1
learning_rate = 0.001
epochs = 7000

model = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
model.train(np.array(X_train), np.array(y_train), epochs)
predictions = model.predict(X_valid)

Epoch 100, Loss: 0.2462
Epoch 200, Loss: 0.2396
Epoch 300, Loss: 0.2354
Epoch 400, Loss: 0.2314
Epoch 500, Loss: 0.2291
Epoch 600, Loss: 0.2280
Epoch 700, Loss: 0.2269
Epoch 800, Loss: 0.2360
Epoch 900, Loss: 0.2326
Epoch 1000, Loss: 0.2333
Epoch 1100, Loss: 0.2269
Epoch 1200, Loss: 0.2222
Epoch 1300, Loss: 0.2242
Epoch 1400, Loss: 0.2110
Epoch 1500, Loss: 0.2214
Epoch 1600, Loss: 0.2263
Epoch 1700, Loss: 0.2006
Epoch 1800, Loss: 0.2085
Epoch 1900, Loss: 0.2078
Epoch 2000, Loss: 0.2081
Epoch 2100, Loss: 0.2039
Epoch 2200, Loss: 0.2165
Epoch 2300, Loss: 0.2153
Epoch 2400, Loss: 0.2302
Epoch 2500, Loss: 0.1929
Epoch 2600, Loss: 0.1743
Epoch 2700, Loss: 0.2404
Epoch 2800, Loss: 0.1613
Epoch 2900, Loss: 0.2216
Epoch 3000, Loss: 0.1627
Epoch 3100, Loss: 0.2138
Epoch 3200, Loss: 0.3052
Epoch 3300, Loss: 0.2530
Epoch 3400, Loss: 0.1540
Epoch 3500, Loss: 0.1546
Epoch 3600, Loss: 0.1576
Epoch 3700, Loss: 0.1629
Epoch 3800, Loss: 0.1510
Epoch 3900, Loss: 0.2124
Epoch 4000, Loss: 0.1618
Epoch 410

### Get Model Accuracy

In [57]:
def validate(y_pred, y_valid):
    num_of_correct_predictions = np.sum((y_pred >= 0.5) == y_valid)
    accuracy = num_of_correct_predictions / len(y_valid)
    return accuracy

y_valid_np = np.array(y_valid)
y_valid_np = y_valid_np.reshape(-1,1)
print(validate(np.array(predictions), y_valid_np))
print(model.validate(np.array(predictions), y_valid_np))

print(np.array(predictions)[10:20])
print(np.array(y_valid)[10:20])

print((np.array(predictions)).shape)
print((np.array(y_valid)).shape)

0.6655473472128945
0.2961719274680994
[[5.66720631e-87]
 [1.00000000e+00]
 [1.00000000e+00]
 [6.96493173e-01]
 [1.04633169e-82]
 [5.50057997e-01]
 [1.00000000e+00]
 [5.48183661e-01]
 [1.00000000e+00]
 [4.63580159e-01]]
[0 1 1 0 1 0 0 0 1 1]
(1489, 1)
(1489,)
